# Notebook to store sales records (JSON) into bronze.sales table

Define constant variables (location, target entity)

In [0]:
%run ../../config/utils

In [0]:
TARGET_TABLE="brz_sales"

### Getting stream data

In [0]:
schema = spark.table(f'{catalog_name}.{bronze_schema_name}.{TARGET_TABLE}').schema
df_sales = (spark.readStream
            .format("json")
            .option("maxFilesPerTrigger", 10)
            .schema(schema)
            # .option("cloudFiles.schemaLocation", checkpoint_path)
            .load(sales_volume_path)
          )

### Write into bronze.sales

In [0]:
(df_sales.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", bronze_checkpoint_path)
    .trigger(availableNow=True)
    .table(f"{catalog_name}.{bronze_schema_name}.{TARGET_TABLE}")
    .awaitTermination()
  )